# Create a smooth fiducial for SOLikeT/MFLike

In this notebook, we generate a fiducial smooth (i.e., binned spectrum-based theoretical prediction, no map-based random realization) dataset. We start from cmb+fg spectra (MFLike).

In [1]:
%matplotlib inline
import os
import tempfile

import camb
import cobaya
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

print("     Numpy :", np.__version__)
print("Matplotlib :", mpl.__version__)
print("      CAMB :", camb.__version__)
print("    Cobaya :", cobaya.__version__)

     Numpy : 1.24.3
Matplotlib : 3.8.0
      CAMB : 1.5.0
    Cobaya : 3.5.4


In [2]:
# Fiducial cosmo params. When used in a MCMC, the smooth data must agree with this input values

# set neutrino masses
m1=0
m2=np.sqrt(7.5e-5)
m3=np.sqrt(2.5e-3)

sumnu=m1+m2+m3

f1=m1/sumnu
f2=m2/sumnu
f3=m3/sumnu

print('sumnu =', sumnu)
print('')
print('f1 =', f1, '\nf2 =', f2, '\nf3 =', f3)

cosmo_params = {
    "cosmomc_theta": 0.0104090,
    "logA": {"value": 3.045, "drop": True},
    "As": {"value": "lambda logA: 1e-10*np.exp(logA)"},
    "ombh2": 0.02236,
    "omch2": 0.1202,
    "ns": 0.9649,
    "Alens": 1.0,
    "tau": 0.0544,
    "mnu": sumnu,
    "nnu": 3.044,
}

fg_params = {
    "a_tSZ": 3.30,
    "a_kSZ": 1.60,
    "a_p": 6.90,
    "beta_p": 2.08,
    "a_c": 4.90,
    "beta_c": 2.20,
    "a_s": 3.10,
    "T_d": 9.60,
    "a_gtt": 2.81,
    "a_gte": 0.10,
    "a_gee": 0.10,
    "a_psee": 0.000,
    "a_pste": 0.000,
    "xi": 0.20,
}

nuisance_params = {
    "bandint_shift_93": 0,  # only ideal values for now
    "bandint_shift_145": 0,
    "bandint_shift_225": 0,
    "cal_LAT_93": 1,
    "cal_LAT_145": 1,
    "cal_LAT_225": 1,
    "calT_LAT_93": 1,
    "calT_LAT_145": 1,
    "calT_LAT_225": 1,
    "calE_LAT_93": 1,
    "calE_LAT_145": 1,
    "calE_LAT_225": 1,
    "calG_all": 1,
    "alpha_LAT_93": 0,
    "alpha_LAT_145": 0,
    "alpha_LAT_225": 0,
}

sumnu = 0.05866025403784439

f1 = 0.0 
f2 = 0.14763410387308012 
f3 = 0.8523658961269198


Let's use SOLikeT/MFLike to generate the fiducial spectra

In [3]:
packages_path = "/Users/bradamante/Library/CloudStorage/GoogleDrive-grbmtn@unife.it/My Drive/Work/Projects/Open/cobaya_packages"

In [4]:
pre = "LAT_simu_sacc_"
info = {
    "likelihood": {
    "mflike.TTTEEE": {
        "package_install": "pip",
        "data_folder": "MFLike/v0.8",
        "input_file": pre + "00000.fits",
        "cov_Bbl_file": "data_sacc_w_covar_and_Bbl.fits",
        "defaults": {"polarizations": ["TT","TE","ET","EE"],
                     "scales": {"TT": [30, 9000],
                                "TE": [30, 9000],
                                "ET": [30, 9000],
                                "EE": [30, 9000]},
                     "symmetrize": False
                    },
    }
},
        "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                           "WantTransfer": True,
                                           "Transfer.high_precision": True,
                                           "Transfer.kmax": 1.2,
                                           "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3
                                           "num_nu_massive": 2,
                                           "nu_mass_eigenstates": 2,
                                           "nu_mass_fractions": [f2,f3],
                                           "nu_mass_numbers": [1,1],
                                           "share_delta_neff": False},
                            "stop_at_error": True}},
        "params": cosmo_params,
        "packages_path": packages_path,
        "debug": True,
}

In [5]:
info["theory"]["mflike.BandpowerForeground"] = {'stop_at_error': True}

In [6]:
from cobaya.model import get_model

model = get_model(info)

 2025-03-11 15:06:53,203 [input] 'mflike.BandpowerForeground' could not be found as internal, trying external.
 2025-03-11 15:06:53,669 [input] 'mflike.TTTEEE' could not be found as internal, trying external.
 2025-03-11 15:06:53,687 [model] Input info updated with defaults (dumped to YAML):
theory:
  camb:
    version: null
    path: null
    speed: 0.3
    stop_at_error: true
    extra_args:
      lens_potential_accuracy: 1
      WantTransfer: true
      Transfer.high_precision: true
      Transfer.kmax: 1.2
      num_nu_massless: 1.044
      num_nu_massive: 2
      nu_mass_eigenstates: 2
      nu_mass_fractions:
      - 0.14763410387308012
      - 0.8523658961269198
      nu_mass_numbers:
      - 1
      - 1
      share_delta_neff: false
    ignore_obsolete: false
    use_renames: false
    external_primordial_pk: false
    renames:
      omegabh2: ombh2
      omegach2: omch2
      omegal: omega_de
      omegak: omk
      yhe: YHe
      yheused: YHe
      YpBBN: Y_p
  mflike.Bandpow

In [7]:
my_mflike = model.likelihood["mflike.TTTEEE"]
chi2 = -2 * (model.loglikes(fg_params)[0] - my_mflike.logp_const)

 2025-03-11 15:06:58,160 [model] Got input parameters: {'cosmomc_theta': 0.010409, 'logA': 3.045, 'As': 2.101003120287951e-09, 'ombh2': 0.02236, 'omch2': 0.1202, 'ns': 0.9649, 'Alens': 1.0, 'tau': 0.0544, 'mnu': 0.05866025403784439, 'nnu': 3.044, 'beta_s': -2.5, 'alpha_s': 1.0, 'T_effd': 19.6, 'beta_d': 1.5, 'a_tSZ': 3.3, 'a_kSZ': 1.6, 'a_p': 6.9, 'beta_p': 2.08, 'a_c': 4.9, 'beta_c': 2.2, 'a_s': 3.1, 'a_gtt': 2.81, 'xi': 0.2, 'T_d': 9.6, 'alpha_dT': -0.6, 'alpha_p': 1.0, 'alpha_tSZ': 0.0, 'a_gte': 0.1, 'a_pste': 0.0, 'alpha_dE': -0.4, 'a_gee': 0.1, 'a_psee': 0.0, 'cal_LAT_93': 1.0, 'cal_LAT_145': 1.0, 'cal_LAT_225': 1.0, 'calG_all': 1.0, 'alpha_LAT_93': 0.0, 'alpha_LAT_145': 0.0, 'alpha_LAT_225': 0.0, 'bandint_shift_LAT_93': 0.0, 'bandint_shift_LAT_145': 0.0, 'bandint_shift_LAT_225': 0.0, 'calT_LAT_93': 1.0, 'calT_LAT_145': 1.0, 'calT_LAT_225': 1.0, 'calE_LAT_93': 1.0, 'calE_LAT_145': 1.0, 'calE_LAT_225': 1.0}
 2025-03-11 15:06:58,167 [camb.transfers] Got parameters {'cosmomc_theta': 

We now store theory CMB and FG Cls and use MFLike functions to manipulate them into a data model

In [8]:
Dl = model.components[2].get_Cl(ell_factor=True)
fg_totals = model.components[3].get_fg_totals()

In [9]:
#need to define this to pass params as a single dictionary to get modified theory below
params = {**cosmo_params,**fg_params}
params = {**params, **nuisance_params}
print(params)

{'cosmomc_theta': 0.010409, 'logA': {'value': 3.045, 'drop': True}, 'As': {'value': 'lambda logA: 1e-10*np.exp(logA)'}, 'ombh2': 0.02236, 'omch2': 0.1202, 'ns': 0.9649, 'Alens': 1.0, 'tau': 0.0544, 'mnu': 0.05866025403784439, 'nnu': 3.044, 'a_tSZ': 3.3, 'a_kSZ': 1.6, 'a_p': 6.9, 'beta_p': 2.08, 'a_c': 4.9, 'beta_c': 2.2, 'a_s': 3.1, 'T_d': 9.6, 'a_gtt': 2.81, 'a_gte': 0.1, 'a_gee': 0.1, 'a_psee': 0.0, 'a_pste': 0.0, 'xi': 0.2, 'bandint_shift_93': 0, 'bandint_shift_145': 0, 'bandint_shift_225': 0, 'cal_LAT_93': 1, 'cal_LAT_145': 1, 'cal_LAT_225': 1, 'calT_LAT_93': 1, 'calT_LAT_145': 1, 'calT_LAT_225': 1, 'calE_LAT_93': 1, 'calE_LAT_145': 1, 'calE_LAT_225': 1, 'calG_all': 1, 'alpha_LAT_93': 0, 'alpha_LAT_145': 0, 'alpha_LAT_225': 0}


In [10]:
mflike = model.components[0]
dls = {s: Dl[s][mflike.l_bpws] for s, _ in mflike.lcuts.items()}
#combine CMB and FG and add svstematics
DlsObs = mflike.get_modified_theory(dls, fg_totals, **params)

We now bin the theory data model

In [11]:
ps_dic = {}
ps_vec = np.zeros_like(model.components[0].data_vec)

for m in model.components[0].spec_meta:
    p = m["pol"]
    ids = m["ids"]
    w = m["bpw"]
    t1 = m["t1"]
    t2 = m["t2"]
    
    #print(p, ids, t1, t2)
    if (t1+"x"+t2 not in ps_dic.keys ()):
        ps_dic[t1+"x"+t2]={"lbin": m["leff"]}
        
    dls_obs = DlsObs[p, m["t2"], m["t1"]] if m["hasYX_xsp"] else DlsObs[p, m["t1"], m["t2"]]
    for i, nonzero, weights in zip (m["ids"], w.nonzeros, w.sliced_weights):
        ps_vec[i] = weights @ dls_obs[nonzero]
        
    ps_dic[t1+"x"+t2].update({p: ps_vec[ids]})

In [12]:
ps_dic.keys() #check that all spectra are here

dict_keys(['LAT_93xLAT_93', 'LAT_93xLAT_145', 'LAT_93xLAT_225', 'LAT_145xLAT_145', 'LAT_145xLAT_225', 'LAT_225xLAT_225'])

We now save the spectra in the format needed by ```sacc```: lbin,TT,TE,TB,ET,BT,EE,EB,BE,BB

In [13]:
namedir=packages_path+"/data/soliket_mflike/smooth_data/split_mflike/"
#namedir="/Users/bradamante/Google Drive/My Drive/Work/Projects/Open/cobaya_packages/data/soliket_mflike/smooth_data/"
for k in ps_dic.keys():
    namefile = "Dl_"+k+"_auto_00000.dat"
    l = ps_dic[k]["lbin"]
    tt = ps_dic[k]["tt"]
    te = ps_dic[k]["te"]
    ee = ps_dic[k]["ee"]
    tbebbb = np.zeros(len(l))
    np.savetxt(namedir+namefile,np.column_stack((l,tt,te,tbebbb,te,tbebbb,ee,tbebbb,tbebbb,tbebbb)))

In [14]:
cltest=np.loadtxt(namedir+namefile)

In [15]:
cltest.shape

(147, 10)

We now use the routine in ``PSPype`` to convert the .txt data in sacc data. We first read the spectra:

In [16]:
data = {}
sim_suffix = "00000"
for spec_name in ps_dic.keys():
    na, nb = spec_name.split("x")
    data[na,nb] = {}
    spec = np.loadtxt("%s/Dl_%s_auto_%s.dat" % (namedir, spec_name, sim_suffix), unpack=True)
    ps = {"lbin": spec[0],
          "TT": spec[1],
          "TE": spec[2],
          "TB": spec[3],
          "ET": spec[4],
          "BT": spec[5],
          "EE": spec[6],
          "EB": spec[7],
          "BE": spec[8],
          "BB": spec[9]}
    data[na,nb] = ps

Then define useful functions

In [17]:
exp_freq = ['LAT_93', 'LAT_145', 'LAT_225']
pols = ["T", "E", "B"]
map_types = {"T": "0", "E": "e", "B": "b"}

def get_x_iterator():
    for id_efa, efa in enumerate(exp_freq):
        for id_efb, efb in enumerate(exp_freq):
            if (id_efa > id_efb): continue
            for ipa, pa in enumerate(pols):
                if (efa == efb):
                    polsb = pols[ipa:]
                else:
                    polsb = pols
                for pb in polsb:
                    yield (efa, efb, pa, pb)
                    print(efa, efb, pa, pb)

Import ```sacc``` and start adding info

In [18]:
import sacc

spec_sacc =  sacc.Sacc()

In [19]:
for exp_f in exp_freq:
    print("%s_s0" % (exp_f))

    my_data_bandpasses = {"nu":np.array([float(exp_f.split("_")[1])]), "b_nu":np.array([1.])}
    my_data_beams = {"l":np.arange(10000), "bl":np.ones(10000)}

    # CMB temperature 
    spec_sacc.add_tracer("NuMap", "%s_s0" % (exp_f),
                                 quantity="cmb_temperature", spin=0,
                                 nu=model.components[0].bands[exp_f+"_s0"]["nu"],
                                 bandpass=model.components[0].bands[exp_f+"_s0"]["bandpass"],
                                 ell=my_data_beams["l"],
                                 beam=my_data_beams["bl"])
    
    # CMB polarization
    spec_sacc.add_tracer("NuMap", "%s_s2" % (exp_f),
                                 quantity="cmb_polarization", spin=2,
                                 nu=model.components[0].bands[exp_f+"_s2"]["nu"],
                                 bandpass=model.components[0].bands[exp_f+"_s2"]["bandpass"],
                                 ell=my_data_beams["l"],
                                 beam=my_data_beams["bl"])

LAT_93_s0
LAT_145_s0
LAT_225_s0


Now add spectra to sacc object and save in fits format

In [20]:
for id_x, (efa, efb, pa, pb) in enumerate(get_x_iterator()):
    if pa == "T":
        ta_name = "%s_s0" % (efa)
    else:
        ta_name = "%s_s2" % (efa)

    if pb == "T":
        tb_name = "%s_s0" % (efb)
    else:
        tb_name = "%s_s2" % (efb)

    if pb == "T":
        cl_type = "cl_" + map_types[pb] + map_types[pa]
    else:
        cl_type = "cl_" + map_types[pa] + map_types[pb]

    lbin = data[efa, efb]["lbin"]
    cb = data[efa, efb][pa + pb]

    spec_sacc.add_ell_cl(cl_type, ta_name, tb_name, lbin, cb)

LAT_93 LAT_93 T T
LAT_93 LAT_93 T E
LAT_93 LAT_93 T B
LAT_93 LAT_93 E E
LAT_93 LAT_93 E B
LAT_93 LAT_93 B B
LAT_93 LAT_145 T T
LAT_93 LAT_145 T E
LAT_93 LAT_145 T B
LAT_93 LAT_145 E T
LAT_93 LAT_145 E E
LAT_93 LAT_145 E B
LAT_93 LAT_145 B T
LAT_93 LAT_145 B E
LAT_93 LAT_145 B B
LAT_93 LAT_225 T T
LAT_93 LAT_225 T E
LAT_93 LAT_225 T B
LAT_93 LAT_225 E T
LAT_93 LAT_225 E E
LAT_93 LAT_225 E B
LAT_93 LAT_225 B T
LAT_93 LAT_225 B E
LAT_93 LAT_225 B B
LAT_145 LAT_145 T T
LAT_145 LAT_145 T E
LAT_145 LAT_145 T B
LAT_145 LAT_145 E E
LAT_145 LAT_145 E B
LAT_145 LAT_145 B B
LAT_145 LAT_225 T T
LAT_145 LAT_225 T E
LAT_145 LAT_225 T B
LAT_145 LAT_225 E T
LAT_145 LAT_225 E E
LAT_145 LAT_225 E B
LAT_145 LAT_225 B T
LAT_145 LAT_225 B E
LAT_145 LAT_225 B B
LAT_225 LAT_225 T T
LAT_225 LAT_225 T E
LAT_225 LAT_225 T B
LAT_225 LAT_225 E E
LAT_225 LAT_225 E B
LAT_225 LAT_225 B B


In [21]:
spec_sacc.save_fits("%s/data_sacc_smooth_%s.fits" % (namedir, sim_suffix), overwrite=True)

# Test new data

Let's read the newly created data and check that we get chi2=0 if compared against same fiducial

In [22]:
pre = "data_sacc_smooth_"
info = {
    "likelihood": {
    "mflike.TTTEEE": {
        "package_install": "pip",
        "data_folder": namedir,
        "input_file": pre + "00000.fits",
        "cov_Bbl_file": packages_path + "/data/MFLike/v0.8/data_sacc_w_covar_and_Bbl.fits",
        "defaults": {"polarizations": ["TT","TE","ET","EE"],
                     "scales": {"TT": [30, 9000],
                                "TE": [30, 9000],
                                "ET": [30, 9000],
                                "EE": [30, 9000]},
                     "symmetrize": False
                    },
    }
},
        "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1,
                                           "WantTransfer": True,
                                           "Transfer.high_precision": True,
                                           "Transfer.kmax": 1.2,
                                           "num_nu_massless": 1.044, #we use the minimal normal ordering case: massless lightest state, m2 and m3
                                           "num_nu_massive": 2,
                                           "nu_mass_eigenstates": 2,
                                           "nu_mass_fractions": [f2,f3],
                                           "nu_mass_numbers": [1,1],
                                           "share_delta_neff": False},
                            "stop_at_error": True}},
        "params": cosmo_params,
        "packages_path": packages_path,
        "debug": True,
}

In [23]:
info["theory"]["mflike.BandpowerForeground"] = {'stop_at_error': True}

In [24]:
from cobaya.model import get_model

model_test = get_model(info)

 2025-03-11 15:07:27,196 [input] 'mflike.BandpowerForeground' could not be found as internal, trying external.
 2025-03-11 15:07:27,230 [input] 'mflike.TTTEEE' could not be found as internal, trying external.
 2025-03-11 15:07:27,255 [model] Input info updated with defaults (dumped to YAML):
theory:
  camb:
    version: null
    path: null
    speed: 0.3
    stop_at_error: true
    extra_args:
      lens_potential_accuracy: 1
      WantTransfer: true
      Transfer.high_precision: true
      Transfer.kmax: 1.2
      num_nu_massless: 1.044
      num_nu_massive: 2
      nu_mass_eigenstates: 2
      nu_mass_fractions:
      - 0.14763410387308012
      - 0.8523658961269198
      nu_mass_numbers:
      - 1
      - 1
      share_delta_neff: false
    ignore_obsolete: false
    use_renames: false
    external_primordial_pk: false
    renames:
      omegabh2: ombh2
      omegach2: omch2
      omegal: omega_de
      omegak: omk
      yhe: YHe
      yheused: YHe
      YpBBN: Y_p
  mflike.Bandpow

In [25]:
my_mflike = model_test.likelihood["mflike.TTTEEE"]
chi2 = -2 * (model_test.loglikes(fg_params)[0] - my_mflike.logp_const)

 2025-03-11 15:07:31,577 [model] Got input parameters: {'cosmomc_theta': 0.010409, 'logA': 3.045, 'As': 2.101003120287951e-09, 'ombh2': 0.02236, 'omch2': 0.1202, 'ns': 0.9649, 'Alens': 1.0, 'tau': 0.0544, 'mnu': 0.05866025403784439, 'nnu': 3.044, 'beta_s': -2.5, 'alpha_s': 1.0, 'T_effd': 19.6, 'beta_d': 1.5, 'a_tSZ': 3.3, 'a_kSZ': 1.6, 'a_p': 6.9, 'beta_p': 2.08, 'a_c': 4.9, 'beta_c': 2.2, 'a_s': 3.1, 'a_gtt': 2.81, 'xi': 0.2, 'T_d': 9.6, 'alpha_dT': -0.6, 'alpha_p': 1.0, 'alpha_tSZ': 0.0, 'a_gte': 0.1, 'a_pste': 0.0, 'alpha_dE': -0.4, 'a_gee': 0.1, 'a_psee': 0.0, 'cal_LAT_93': 1.0, 'cal_LAT_145': 1.0, 'cal_LAT_225': 1.0, 'calG_all': 1.0, 'alpha_LAT_93': 0.0, 'alpha_LAT_145': 0.0, 'alpha_LAT_225': 0.0, 'bandint_shift_LAT_93': 0.0, 'bandint_shift_LAT_145': 0.0, 'bandint_shift_LAT_225': 0.0, 'calT_LAT_93': 1.0, 'calT_LAT_145': 1.0, 'calT_LAT_225': 1.0, 'calE_LAT_93': 1.0, 'calE_LAT_145': 1.0, 'calE_LAT_225': 1.0}
 2025-03-11 15:07:31,579 [camb.transfers] Got parameters {'cosmomc_theta': 

In [26]:
chi2

array([-0.])

Awesome!!!